[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1XRD_FqKjHIsqqDrjkho6sSZEgFLCDGtu#scrollTo=HB3qzAnz1Etv)



This is the google colab notebook for training/retraining the PointNet model for different Bravais Lattices. There are 5 different models corresponding to the different crystal systems (triclinic and hexagonal only contain the primitive lattice).


Please eneter which crystal system is to be trained.

Choose from: Cubic, Trigonal, Monoclinic, Tetragonal, Orthorhombic

In [ ]:
CrySys = 'Cubic'

# Preamble

In [ ]:
#@title Paths and Filenames

# # Optional mount and work with google drive
# from google.colab import drive
# drive.mount('/content/gdrive')

# path = '/content/gdrive/MyDrive/Colab Notebooks/StructurePrediction/'
# weights_path = path + 'Weights/'
# data_path = path + 'DataSets/'


# For use with Google Colob
%cd /content
!rm -rf crystal_structure_prediction
!git clone --single-branch --depth=1 https://github.com/adrianwalsh1990/crystal_structure_prediction.git
%cd crystal_structure_prediction


weights_path = '/content/crystal_structure_prediction/Weights/'
data_path = '/content/crystal_structure_prediction/DataSets/'
filename = CrySys + '.json'
fname = data_path + filename

model_name = weights_path + 'BRAVIS_' + CrySys


/content
Cloning into 'crystal_structure_prediction'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 94 (delta 38), reused 90 (delta 38), pack-reused 0
Unpacking objects: 100% (94/94), done.
Checking out files: 100% (90/90), done.
/content/crystal_structure_prediction


In [ ]:
#@title Import libraries and  mount drive
!pip install trimesh
import pandas as pd
import trimesh
import numpy as np
import tensorflow as tf
import os
import glob
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.activations import relu
from tensorflow.python.keras.backend import categorical_crossentropy
from tensorflow.python.keras.metrics import accuracy
from tensorflow.python.keras.metrics import AUC 
from tensorflow.python.keras.metrics import FalsePositives
from tensorflow.python.keras.metrics import FalseNegatives
from tensorflow.python.keras.metrics import TruePositives
from tensorflow.python.keras.metrics import TrueNegatives
from tensorflow.python.keras.optimizer_v1 import adam
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt



# Prepare the data

The atomic positions from all samples are read from the data file to a numpy array. These points are padded by repeating the first atomic site, until all inputs are of the same length.

In [ ]:
print(fname)

/content/crystal_structure_prediction/DataSets/Cubic.json


In [ ]:
#@title Prepare the sites datapoints from the json file.
# Read in atomic positions

file = pd.read_json(fname)
x = file['structure.sites.xyz'].to_numpy()

# Max number of atomic sites
num_sites = file['nsites'].to_numpy()
NUM_POINTS = np.amax(num_sites)


# Pad the atomic positions until all have the same number of sites
x_padded = np.array([xi + [xi[0]] * (NUM_POINTS - len(xi)) for xi in x])

The labels are all the samples are set using the space groups. The choice of CrySys above determines which Crystal System is being trained. Therefore all space group labels will be converted to a number corresponding to which type of Bravais Lattice is present

In [ ]:
#@title Prepare the output, i.e. using the spacegroup number to indicate the presence of particular pointgroups

y = file['spacegroup.number'].to_numpy()


# Space group to corresponding Bravais Lattices

Brav_P = [1,2,3,4,6,7,10,11,13,14,16,17,18,19,25,26,27,28,29,30,31,32,33,34,47,
          48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,75,76,77,78,81,83,84,85,86,
          89,90,91,92,93,94,95,96,99,100,101,102,103,104,105,106,111,112,113,114,
          115,116,117,118,123,124,125,126,127,128,129,130,131,132,133,134,135,136,
          137,138,143,144,145,146,147,149,150,151,152,153,154,156,157,158,159,162,
          163,164,165,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,
          183,184,185,186,187,188,189,190,191,192,193,194,195,198,200,201,205,207,
          208,212,213,215,218,221,222,223,224]

Brav_I = [23,24,44,45,46,71,72,73,74,79,80,82,87,88,97,98,107,108,109,110,119,120,121,
          122,139,140,141,142,197,199,204,206,211,214,217,220,229,230]

Brav_F = [22,42,43,69,70,196,202,203,209,210,216,219,225,226,227,228]

Brav_A = [38,39,40,41]

Brav_C = [5,8,9,12,15,20,21,25,26,27,35,36,37,63,64,65,66,67,68]

Brav_R = [146,148,155,160,161,166,167]

# Convert the space groups to numerical values for the
# bravais lattice. The reason for the seemingly overlap 
# is due to needing the correct number of outputs for the model
if CrySys =='Trigonal':
  NUM_CLASSES = 2
  for i in list(set(y)):
      if i in Brav_P:
        y[y == i] = 0
      elif i in Brav_R:
        y[y == i] = 1

elif CrySys =='Cubic':
  NUM_CLASSES = 3
  for i in list(set(y)):
      if i in Brav_P:
        y[y == i] = 0
      elif i in Brav_I:
        y[y == i] = 1
      elif i in Brav_F:
        y[y == i] = 2

elif CrySys =='Monoclinic':
  NUM_CLASSES = 2
  for i in list(set(y)):
      if i in Brav_P:
        y[y == i] = 0
      elif i in Brav_C:
        y[y == i] = 1

elif CrySys =='Orthorhombic':
  NUM_CLASSES = 5
  for i in list(set(y)):
      if i in Brav_P:
        y[y == i] = 0
      elif i in Brav_I:
        y[y == i] = 1
      elif i in Brav_F:
        y[y == i] = 2
      elif i in Brav_A:
        y[y == i] = 3
      elif i in Brav_C:
        y[y == i] = 4

elif CrySys =='Tetragonal':
  NUM_CLASSES = 2
  for i in list(set(y)):
      if i in Brav_P:
        y[y == i] = 0
      elif i in Brav_I:
        y[y == i] = 1
      

      


Prepare the train/test datasets, which are coverted to tensors. The augmentation function is designed to jitter and shuffle the train dataset.


In [ ]:
# @title Create the train/test datasets

train_points, test_points, train_labels, test_labels = train_test_split(x_padded, y, test_size=0.1)

def augment(points, label):
    # jitter points
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    # shuffle points
    points = tf.random.shuffle(points)
    return points, label


BATCH_SIZE = 128

train_points = tf.convert_to_tensor(train_points)
test_points = tf.convert_to_tensor(test_points)


train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)

# Create and train the Model

In [ ]:
# @title Create the model

# Example modified from the Keras Code Example 
# https://keras.io/examples/vision/pointnet/

def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

def tnet(inputs, num_features):
    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

inputs = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)


x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
# model.summary()

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["sparse_categorical_accuracy"],
)

Note if OOM errors occur here, consider reducing the BATCH_SIZE above, as this depends on the colab resource allocation available.


In [ ]:
# @title Train new model
model.fit(train_dataset, epochs=100, validation_data=test_dataset)

model.save_weights(model_name)

results = model.evaluate(test_points, test_labels)


Epoch 1/100
 31/147 [=====>........................] - ETA: 23s - loss: 8.1383 - sparse_categorical_accuracy: 0.6862

KeyboardInterrupt: ignored